### 用HongKong的数据做实验

In [51]:
import networkx as nx
import string

In [133]:
with open("D:/Data/quanturb-metro-network-data-cities-2009/HongKong-2009-adjacency.net") as f:
    data =  f.readlines()

In [8]:
data[2].split(" ")

['50', '"CausewayBay"', '22.28021', '114.183482', '0\n']

In [132]:
print string.atoi(data[2].split(" ")[0], 10)
print string.atof(data[2].split(" ")[2])

50
22.28021


In [24]:
data[88].split(" ")

['7', '8', '1\n']

In [134]:
stopdict = {}
G = nx.Graph()
for i in data:
    if i.split(" ")[-1] == '0\n': # The stop lines are ended with 0
        stop_id = i.split(" ")[0]
        stop_name = i.split(" ")[1][1:-1]
        stop_lat = i.split(" ")[2]
        stop_lon = i.split(" ")[3]
        stopdict[stop_id] = [stop_name, stop_lat, stop_lon]
    if i.split(" ")[-1] == '1\n': # The edge lines are ended with 1
        start = i.split(" ")[0]
        end = i.split(" ")[1]
        G.add_edge(start, end)
print nx.info(G)

Name: 
Type: Graph
Number of nodes: 82
Number of edges: 85
Average degree:   2.0732


In [165]:
def calculate_e_global(G):
    avg = 0.0
    for node in G:
        path_length_list=nx.single_source_shortest_path_length(G, node)
        for e in path_length_list.values():
            if e != 0:
                avg += 1.0/e
            else:
                avg += 0
    n=len(G)
    if n != 1:
        avg = avg/(n*(n-1))
    else:
        avg += 0
    return avg
    
def calculate_e_local(G):
    avg = 0.0
    for node in G:
        avg += calculate_e_global(G.subgraph(nx.bfs_successors(G,node)[node]))
    avg = avg/len(G)
    return avg

In [61]:
calculate_e_global(G)

0.15024595825121367

In [62]:
calculate_e_local(G)

0.006097560975609756

### 计算其他城市地铁的efficiency（unweighted）

In [149]:
def calculate_e_global(G):
    avg = 0.0
    for node in G:
        path_length_list=nx.single_source_shortest_path_length(G, node)
        for e in path_length_list.values():
            if e != 0:
                avg += 1.0/e
            else:
                avg += 0
    n=len(G)
    if n != 1:
        avg = avg/(n*(n-1))
    else:
        avg += 0
    return avg
    
def calculate_e_local(G):
    avg = 0.0
    for node in G:
        avg += calculate_e_global(G.subgraph(nx.bfs_successors(G,node)[node]))
    avg = avg/len(G)
    return avg

def calculate_city_subway_efficiency(cityName):
    with open("D:/Data/quanturb-metro-network-data-cities-2009/" + cityName + "-2009-adjacency.net") as f:
        data =  f.readlines()
    stopdict = {}
    G = nx.Graph()
    for i in data:
        if i.split(" ")[-1] == '0\n': # The stop lines are ended with 0
            stop_id = i.split(" ")[0]
            stop_name = i.split(" ")[1][1:-1]
            stop_lat = i.split(" ")[2]
            stop_lon = i.split(" ")[3]
            stopdict[stop_id] = [stop_name, stop_lat, stop_lon]
        if i.split(" ")[-1] == '1\n': # The edge lines are ended with 1
            start = i.split(" ")[0]
            end = i.split(" ")[1]
            G.add_edge(start, end)
    print cityName, " Eglob = ", calculate_e_global(G)
    print cityName, " Eloc = ",calculate_e_local(G)

In [130]:
calculate_city_subway_efficiency("Madrid")
calculate_city_subway_efficiency("Chicago")
calculate_city_subway_efficiency("Berlin")
calculate_city_subway_efficiency("Beijing")
calculate_city_subway_efficiency("Barcelona")
calculate_city_subway_efficiency("NewYork")
calculate_city_subway_efficiency("Shanghai")
calculate_city_subway_efficiency("Tokyo")
calculate_city_subway_efficiency("Paris")
calculate_city_subway_efficiency("Seoul")
calculate_city_subway_efficiency("Moscow")
calculate_city_subway_efficiency("Mexico")
calculate_city_subway_efficiency("Osaka")

Madrid  Eglob =  0.120476760921
Madrid  Eloc =  0.00614035087719
Chicago  Eglob =  0.114162607041
Chicago  Eloc =  0.0182033096927
Berlin  Eglob =  0.109005791837
Berlin  Eloc =  0.00588235294118
Beijing  Eglob =  0.149773533382
Beijing  Eloc =  0.0201923076923
Barcelona  Eglob =  0.14413603838
Barcelona  Eloc =  0.008984375
NewYork  Eglob =  0.0732230925809
NewYork  Eloc =  0.017277026284
Shanghai  Eglob =  0.119685146129
Shanghai  Eloc =  0.00292792792793
Tokyo  Eglob =  0.137223621792
Tokyo  Eloc =  0.0236705434862
Paris  Eglob =  0.116207527235
Paris  Eloc =  0.0215320910973
Seoul  Eglob =  0.0762680146974
Seoul  Eloc =  0.00595238095238
Moscow  Eglob =  0.151920095643
Moscow  Eloc =  0.0195273631841
Mexico  Eglob =  0.134071376436
Mexico  Eloc =  0.00340136054422
Osaka  Eglob =  0.162369801757
Osaka  Eloc =  0.0


### 加入权重后计算efficiency

In [171]:
from math import *
import networkx as nx
import string

def dis(lat1,lon1,lat2,lon2):
    R = 6373.0 # convert Earth's radius in kilometers
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

def calculate_e_global_with_weight(G, stopdict):
    avg = 0.0
    d = 0.0
    for node in G:
        # logging.debug("node = ", node)
        path_length_list=nx.single_source_shortest_path_length(G, node)
        # logging.debug("path_length_list: ", path_length_list)
        for i in path_length_list.keys():
            if path_length_list[i] != 0:
                x = node
                y = i
                l = dis(string.atof(stopdict[x][1]), string.atof(stopdict[x][2]), string.atof(stopdict[y][1]), string.atof(stopdict[y][2]))
                a = path_length_list[i]
                d = a * l
                # logging.debug("x = ", x, "y = ", y, "l = ", l, "a = ", a, "d = ", d)
            if d != 0:
                avg += 1.0/d
            else:
                avg += 0
    n=len(G)
    if n != 1:
        avg = avg/(n*(n-1))
    else:
        avg += 0
    return avg
    
def calculate_e_local_with_weight(G, stopdict):
    avg = 0.0
    for node in G:
        avg += calculate_e_global_with_weight(G.subgraph(nx.bfs_successors(G,node)[node]), stopdict)
    avg = avg/len(G)
    return avg

def calculate_city_subway_efficiency_with_weight(cityName):
    with open("D:/Data/quanturb-metro-network-data-cities-2009/" + cityName + "-2009-adjacency.net") as f:
        data =  f.readlines()
    stopdict = {}
    G = nx.Graph()
    for i in data:
        if i.split(" ")[-1] == '0\n': # The stop lines are ended with 0
            stop_id = i.split(" ")[0]
            stop_name = i.split(" ")[1][1:-1]
            stop_lat = i.split(" ")[2]
            stop_lon = i.split(" ")[3]
            stopdict[stop_id] = [stop_name, stop_lat, stop_lon]
        if i.split(" ")[-1] == '1\n': # The edge lines are ended with 1
            start = i.split(" ")[0]
            end = i.split(" ")[1]
            G.add_edge(start, end)
    print cityName, " Eglob = ", calculate_e_global_with_weight(G, stopdict)
    print cityName, " Eloc = ",calculate_e_local_with_weight(G, stopdict)

In [142]:
calculate_city_subway_efficiency_with_weight("NewYork")

NewYork  Eglob =  0.04596312624
NewYork  Eloc =  0.0108450232318


In [170]:
calculate_city_subway_efficiency_with_weight("NewYork")

NewYork  Eglob =  0.0468077440649
NewYork  Eloc =  0.0424931083167
